In [1]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import os
import requests
import time
from shapely.validation import make_valid
from shapely.ops import transform
from shapely.geometry import MultiPolygon
from cbers4asat import Cbers4aAPI
import pyproj
import cv2

In [3]:
shp_path = "C:/ .shp" #shp da AOI
out_dir = 'C:/ '  # diretório para saída dos arquivos
user_cbers = ''  # email cadastrado no site de catálogos do INPE/CBERS
scene_id = 'CBERS4A_WPM21211420240911'  # Informe o ID da cena aqui - produto WPM_L4_DN
buffer_size = 7000   #tamanho do buffer em metros a partir da AOI para recorte das imagens 
epsg_dest = 4326  # Exemplo de EPSG, 4326 (WGS84) ou 32723 (UTM Brasil)

In [4]:
def corrigir_geometria(geom):
    """Corrige a geometria para garantir que seja válida"""
    if not geom.is_valid:
        geom = make_valid(geom)
    if not geom.is_valid:
        geom = geom.buffer(0)  # Buffer pequeno para corrigir geometria
    return geom

def buffer_geometry(geom, buffer_size):
    """Aplica um buffer à geometria"""
    proj_utm = pyproj.CRS.from_epsg(32723)  # Defina o EPSG correto, se necessário
    proj_wgs84 = pyproj.CRS.from_epsg(4326)
    transformer_to_utm = pyproj.Transformer.from_crs(proj_wgs84, proj_utm, always_xy=True)
    transformer_to_wgs84 = pyproj.Transformer.from_crs(proj_utm, proj_wgs84, always_xy=True)
    geom_utm = transform(transformer_to_utm.transform, geom)
    buffer_utm = corrigir_geometria(geom_utm.buffer(buffer_size))
    return transform(transformer_to_wgs84.transform, buffer_utm)

# Carregar shapefile e re-projetá-lo para o EPSG desejado (sem precisar de arquivo EPSG)
gdf = gpd.read_file(shp_path)

# Verificar se o CRS está definido e atribuir um CRS caso não esteja
if gdf.crs is None:
    print("CRS não definido. Atribuindo CRS manualmente para EPSG:4326...")
    gdf.set_crs(epsg=4326, allow_override=True, inplace=True)  # Atribui EPSG:4326 caso não tenha CRS
else:
    print(f"CRS original: {gdf.crs}")

# Converter para o EPSG desejado (exemplo EPSG:4326)
gdf_wgs84 = gdf.to_crs(epsg=epsg_dest)

# Corrigir geometria e aplicar buffer
perimetro = corrigir_geometria(gdf_wgs84.union_all())
buffered_perimeter = buffer_geometry(perimetro, buffer_size)

# Exemplo de uso do API do CBERS para obter imagens, processar e salvar (baseado no código original)
api = Cbers4aAPI(user_cbers)
produto = api.query_by_id(scene_id=scene_id, collection="CBERS4A_WPM_L4_DN")

if produto:
    image_date = produto['features'][0]['properties']['datetime'][:10]
    assets = produto['features'][0]['assets']
    bands = ['blue', 'green', 'red', 'nir', 'pan']
    urls = {band: assets[band]['href'] for band in bands}

    def download_file(url, dest_path, user, max_retries=3):
        """Função para download de arquivos com tentativas em caso de falha"""
        for attempt in range(max_retries):
            try:
                response = requests.get(url, params={'email': user}, stream=True)
                if response.status_code == 200:
                    with open(dest_path, 'wb') as file:
                        for chunk in response.iter_content(chunk_size=8192):
                            file.write(chunk)
                    return True
            except:
                time.sleep(2)
        return False

    for band, url in urls.items():
        dest_path = os.path.join(out_dir, f'{scene_id}_{band}.tif')
        download_file(url, dest_path, user_cbers)

    cropped_dir = os.path.join(out_dir, "cropped")
    final_path = os.path.join(out_dir, "FINAL_RESULT")
    os.makedirs(final_path, exist_ok=True)
    os.makedirs(cropped_dir, exist_ok=True)

    def recortar_imagem(image_path, geom, out_path):
        """Função para recortar imagem usando a geometria definida"""
        with rasterio.open(image_path) as src:
            geom_reprojected = gpd.GeoSeries([geom], crs='epsg:4326').to_crs(src.crs)
            out_image, out_transform = mask(src, geom_reprojected.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({"height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform, "count": src.count})
            with rasterio.open(out_path, "w", **out_meta) as dest:
                dest.write(out_image)

    cropped_paths = {}
    for band in bands:
        cropped_band_path = os.path.join(cropped_dir, f'{scene_id}_{band}_cropped.tif')
        recortar_imagem(os.path.join(out_dir, f'{scene_id}_{band}.tif'), buffered_perimeter, cropped_band_path)
        cropped_paths[band] = cropped_band_path

    def resize_raster(src_array, src_transform, src_crs, target_shape, target_transform):
        """Função para redimensionar raster"""
        dst_array = np.empty(target_shape, dtype=src_array.dtype)
        reproject(
            source=src_array,
            destination=dst_array,
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=target_transform,
            dst_crs=src_crs,
            resampling=Resampling.bilinear
        )
        return dst_array

    def RCS(multi, pan):
        """Fusão de imagens PAN e Multiespectral"""
        filtro = cv2.blur(pan, (5, 5))
        filtro[filtro == 0] = 1e-6  # evitar divisão por zero
        fusionado = np.zeros(multi.shape, dtype=np.float32)
        for i in range(3):
            fusionado[:, :, i] = (multi[:, :, i] / filtro) * pan
        return np.clip(fusionado, 0, 65535).astype(np.uint16)

    # Processar imagens e salvar resultados
    with rasterio.open(cropped_paths["red"]) as red_src:
        profile = red_src.profile
        profile.update(count=3)

        rgb_stack = np.stack([rasterio.open(cropped_paths[c]).read(1) for c in ["red", "green", "blue"]], axis=-1)
        nir_stack = np.stack([rasterio.open(cropped_paths[c]).read(1) for c in ["nir", "green", "blue"]], axis=-1)
        pan_data = rasterio.open(cropped_paths["pan"]).read(1)

        pan_shape = pan_data.shape
        pan_transform = rasterio.open(cropped_paths["pan"]).transform
        rgb_resized = np.zeros((pan_shape[0], pan_shape[1], 3), dtype=np.float32)
        nirgb_resized = np.zeros((pan_shape[0], pan_shape[1], 3), dtype=np.float32)

        for i in range(3):
            rgb_resized[:, :, i] = resize_raster(rgb_stack[:, :, i], red_src.transform, red_src.crs, pan_shape, pan_transform)
            nirgb_resized[:, :, i] = resize_raster(nir_stack[:, :, i], red_src.transform, red_src.crs, pan_shape, pan_transform)

        pansharp_rgb = RCS(rgb_resized, pan_data.astype(np.float32))
        pansharp_nirgb = RCS(nirgb_resized, pan_data.astype(np.float32))

        # Salvar RGB simples
        rgb_path = os.path.join(final_path, f"CBERS4A_RGB_{image_date}.tif")
        with rasterio.open(rgb_path, 'w', **profile) as dst:
            for i in range(3):
                dst.write(rgb_stack[:, :, i], i+1)

        # Salvar NIR-GB
        nirgb_path = os.path.join(final_path, f"CBERS4A_NIRGB_{image_date}.tif")
        with rasterio.open(nirgb_path, 'w', **profile) as dst:
            for i in range(3):
                dst.write(nir_stack[:, :, i], i+1)

        # Salvar PAN-RGB
        pan_rgb_path = os.path.join(final_path, f"CBERS4A_PAN_RGB_RCS_{image_date}.tif")
        profile.update(height=pan_shape[0], width=pan_shape[1], transform=pan_transform)
        with rasterio.open(pan_rgb_path, 'w', **profile) as dst:
            for i in range(3):
                dst.write(pansharp_rgb[:, :, i], i+1)

        # Salvar PAN-NIRGB
        pan_nirgb_path = os.path.join(final_path, f"CBERS4A_PAN_NIRGB_RCS_{image_date}.tif")
        with rasterio.open(pan_nirgb_path, 'w', **profile) as dst:
            for i in range(3):
                dst.write(pansharp_nirgb[:, :, i], i+1)

        # Calcular NDVI
        red = rasterio.open(cropped_paths["red"]).read(1).astype(np.float32)
        nir = rasterio.open(cropped_paths["nir"]).read(1).astype(np.float32)
        ndvi = (nir - red) / (nir + red + 1e-6)
        ndvi = np.clip(ndvi, -1, 1)
        profile.update(count=1, dtype='float32')
        ndvi_path = os.path.join(final_path, f"CBERS4A_NDVI_{image_date}.tif")
        with rasterio.open(ndvi_path, 'w', **profile) as dst:
            dst.write(ndvi, 1)

    print(f"Imagens salvas em {final_path}")
else:
    print("Produto não encontrado.")

CRS original: EPSG:4674
Imagens salvas em C:/Users/julio/OneDrive - caputiavaliacoes.com.br/Arquivos de Marina Diotto - GEO/Teste_Automatiza/codigo_imagens/CBERS/CBERS_output\FINAL_RESULT
